<a href="https://www.kaggle.com/code/andreastsimerikas/pneumonia-classification?scriptVersionId=95314242" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

traindf = pd.read_csv('../input/pneumoniasized/labels_train.csv', dtype=str)
# path for the square images with black padding 
train_path = '../input/pneumoniasized/sized_train'

# Class_0 = Normal, Class_1 = Bacterial-Pneumonia, Class_2 = Viral-Pneumonia
# Splitting the main dataframe into smaller class spesific dataframes 
class_0=traindf.loc[traindf.class_id == '0']
class_0.reset_index(drop=True,inplace=True)
class_1=traindf.loc[traindf.class_id == '1']
class_1.reset_index(drop=True,inplace=True)
class_2=traindf.loc[traindf.class_id == '2']
class_2.reset_index(drop=True,inplace=True)

# Combining a portion of our class dataframes into one dataframe for training, maintaining the distribution 
train = class_0[:920]
train = train.append(class_1[:1678])
train = train.append(class_2[:905])
train.reset_index(drop=True,inplace=True)

# Combining the rest of our class dataframes into one dataframe for testing
test = class_0[920:]
test = test.append(class_1[1678:])
test = test.append(class_2[905:])
test.reset_index(drop=True,inplace=True)

# Define the number of splits for cross-validation
split_num = 3
kfold = StratifiedKFold(n_splits=split_num, shuffle=True)

VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []

In [ ]:
from matplotlib import pyplot as plt

# Visualizing the distribution of our dataset
plt.hist(train['class_id'],rwidth=0.8, bins=3, histtype='bar', label='Classes Distribution')

In [ ]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt

path='../input/pneumoniasized/sized_train/'

fig = plt.figure(figsize=(20, 20))
  
# setting values to rows and column variables
rows = 1
columns = 3

# image indexes to select
index_1=0
index_2=1700
index_3=3200
  
# reading images
Image1 = cv2.imread(path+train.iloc[index_1][0],1)
Image1_label = train.iloc[index_1][1]
Image2 = cv2.imread(path+train.iloc[index_2][0],1)
Image2_label = train.iloc[index_2][1]
Image3 = cv2.imread(path+train.iloc[index_3][0],1)
Image3_label = train.iloc[index_3][1]
  
# Adds a subplot at the 1st position
fig.add_subplot(rows, columns, 1)
  
# showing normal case image 
plt.imshow(Image1)
plt.axis('off')
plt.title(Image1_label)
  
# Adds a subplot at the 2nd position
fig.add_subplot(rows, columns, 2)
  
# showing bacterial pneumonia image
plt.imshow(Image2)
plt.axis('off')
plt.title(Image2_label)
  
# Adds a subplot at the 3rd position
fig.add_subplot(rows, columns, 3)
  
# showing viral pneumonia image
plt.imshow(Image3)
plt.axis('off')
plt.title(Image3_label)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, average, BatchNormalization
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.initializers import *
from tensorflow.keras.applications.efficientnet import preprocess_input

img_size = 300

# Creates and returns an EfficientNetB3 model with different classification layers
def create_model(input_shape, n_classes):
    
    # Input and preprocess needed for EfficientNetB3
    inputs = Input(input_shape)
    prep = preprocess_input(inputs)

    # EfficientNetB3 without top classification layers
    conv_base = EfficientNetB3(include_top=False,
                            weights='imagenet',
                            input_tensor=prep)
    
    # Frezzing all BatchNormalization layers
    for layer in conv_base.layers:
        if isinstance(layer, BatchNormalization):
            layer.trainable=False
    # Frezzing the first two blocks
    for layer in conv_base.layers[:73]:       
        layer.trainable=False

    # Top classification layers        
    top_model = GlobalAveragePooling2D()(conv_base.output)
    top_model = Dense(512, activation='relu', kernel_initializer="glorot_normal")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(128, activation='relu', kernel_initializer="glorot_normal")(top_model)
    output = Dense(n_classes, activation='softmax')(top_model)
    
    model = Model(inputs, output)
    
    return model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, average
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.initializers import *
from tensorflow.keras.applications.densenet import preprocess_input

img_size = 224

# Creates and returns a DenseNet121 model with different top classification layers
def create_model(input_shape, n_classes):
    
    # Input and preprocess needed for DenseNet201
    inputs = Input(input_shape)
    prep = preprocess_input(inputs)

    # DenseNet201 without top classification layers and trainable weights
    conv_base = DenseNet201(include_top=False,
                            weights='imagenet',
                            input_tensor=prep)

    # Top classification layers      
    top_model = GlobalAveragePooling2D()(conv_base.output)
    top_model = Dense(512, activation='relu', kernel_initializer="glorot_normal")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(128, activation='relu', kernel_initializer="glorot_normal")(top_model)
    output = Dense(n_classes, activation='softmax')(top_model)
    
    model = Model(inputs, output)
    
    return model

In [ ]:
from tensorflow.keras.utils import plot_model

# Dummy model for visualization 
model = create_model((100,100,3), 3)
plot_model(model)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Different ImageDataGenerators for train and validation batches without live augmentaion
tdatagen = ImageDataGenerator()
vdatagen = ImageDataGenerator()  

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.backend import clear_session
from matplotlib import pyplot as plt

save_dir = './'
fold_no = 1
batch_size = 10
n_classes = 3
n_epochs = 7
fold_no = 1
input_shape=(img_size,img_size,3)


# Splitting image names and labels for kfold cross-validation
imgs = train[['file_name']]
labels = train[['class_id']]

# Returns current fold's model
def get_model_name(num):
    return 'model_'+str(num)+'.h5'

# Training with stratified k-fold cross-validation
for train_index, val_index in kfold.split(imgs,labels):

    # Creating current fold dataframes for training and validation
    fold_train_df = train.iloc[train_index]
    fold_valid_df = train.iloc[val_index]
    
    # Creating train and validation batches using the ImageDataGenerators and the dataframes we created previously
    train_batches = tdatagen.flow_from_dataframe(fold_train_df, 
                                                directory=train_path,
                                                x_col="file_name", 
                                                y_col="class_id",
                                                batch_size=batch_size, 
                                                class_mode="categorical", 
                                                target_size=(img_size,img_size))

    valid_batches = vdatagen.flow_from_dataframe(fold_valid_df, 
                                                directory=train_path,
                                                x_col="file_name", 
                                                y_col="class_id",
                                                batch_size=batch_size, 
                                                class_mode="categorical", 
                                                target_size=(img_size,img_size))

    # Creating the model with our top classification layers  
    model = create_model(input_shape, n_classes)
    model._name="model_"+str(fold_no)

    # Compiling the model with chosen optimizer, loss and metrics
    model.compile(optimizer=Adam(learning_rate=1e-4), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    #-----------------------callbacks----------------------------
    from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

    # Returns leraning rate based on epoch
    def scheduler(epoch, lr):
        if epoch > 2:
            lr=1e-5
            print(lr)
            return lr
        else:
            return lr
        
    lr_scheduler = LearningRateScheduler(scheduler)

    # Saves best model of every fold based on maximum validation accuracy
    model_checkpoint= ModelCheckpoint(filepath="model_"+str(fold_no)+".h5",
                                      monitor='val_accuracy',
                                      mode='max',
                                      save_best_only=True,
                                      verbose=1)
    # List of all our callbacks
    callbacks_list= [model_checkpoint,lr_scheduler]
    #-----------------------callbacks----------------------------
    
    # Creates balanced class weights 
    weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_batches.classes), y=train_batches.classes)
    cw = dict(zip(np.unique(train_batches.classes), weights))
    print('Class weights: ' + str(cw))
    
    # Training using train and validation batches, our callbacks and class weights
    history = model.fit(train_batches,
                        epochs=n_epochs,
                        validation_data=valid_batches,
                        callbacks=callbacks_list,
                        class_weight=cw,
                        verbose=1)  
    
    # Plot of current's fold accuracy, loss, validation accuracy and validation loss
    %matplotlib inline
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('fold '+str(fold_no))
    plt.ylabel('accuracy/loss')
    plt.xlabel('epoch')
    plt.legend(['train_acc', 'val_acc', 'train_loss', 'val_loss'], loc='best')
    plt.grid(visible=True)
    plt.show()  

    model.load_weights("./model_"+str(fold_no)+".h5")
    
    # Fold's saved model evaluation on current's fold validation set
    results = model.evaluate(valid_batches) 
    results = dict(zip(model.metrics_names,results))

    # Appending the results of the fold 
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDAITON_LOSS.append(results['loss'])  

    clear_session()
    fold_no += 1 

In [ ]:
from statistics import mean
from tensorflow.keras.models import load_model

nets = split_num
models = [0] *nets

# List with all the fold models
for j in range(nets):
    models[j] = load_model("./model_"+str(j+1)+".h5")
    
# Mean of all the folds
print(mean(VALIDATION_ACCURACY))
print(mean(VALIDAITON_LOSS))    

In [ ]:
# Creates and returns the average ensemble model of all the folds
def ensemble(models, model_input):
    
    Models_output=[model(model_input) for model in models]
    Avg = average(Models_output)
    
    modelEnsemble = Model(inputs=model_input, outputs=Avg, name='ensemble')
    modelEnsemble.compile(Adam(learning_rate=1e-5), 
                          loss='categorical_crossentropy', 
                          metrics=['accuracy'])
    modelEnsemble.summary()
    
    return modelEnsemble

# Input layer for the ensemble
model_input = Input(shape=models[0].input_shape[1:])
# Creates the ensemble model
ensemble_model = ensemble(models, model_input)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import load_model

# ImageDataGenerator without augmentaions for testing
test_datagen = ImageDataGenerator()

# Test batches with test ImageDataGenerator using our test dataframe
test_batches = test_datagen.flow_from_dataframe(
    dataframe=test,
    directory=train_path,
    x_col='file_name',
    y_col='class_id',
    batch_size=10,
    class_mode='categorical',
    shuffle=False,
    target_size=(img_size,img_size)
)

# loading and evaluating all our fold models on the test data
for j in range(nets):
    models[j] = load_model("./model_"+str(j+1)+".h5")
    print("model_"+str(j+1))
    models[j].evaluate(test_batches)

# evaluating the average ensemble model on the test data    
print("average ensemble")
results = ensemble_model.evaluate(test_batches)
predictions = ensemble_model.predict(test_batches,verbose=1)

# plotting a confusion matrix from the ensemble model predictions
cm = confusion_matrix(test_batches.classes, np.argmax(predictions, axis=-1))
labels = ['0','1','2']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
from sklearn import metrics
import scikitplot as skplt

# ensemble model classification report with precision, recall, f1-score and accuracy
y = np.argmax(predictions, axis=-1)
cr = classification_report(y_true=test_batches.classes, y_pred=y, target_names=test_batches.class_indices)
print(cr)

# roc curves of all the classes
skplt.metrics.plot_roc(test_batches.classes, predictions)
plt.show()

In [ ]:
import os
from tensorflow.keras.models import load_model

test_datagen = ImageDataGenerator()
test_path = '../input/pneumonia/sized/sized_test_df'
test_images = []

i=1
for img in os.listdir(test_path):
    test_images.append(img)
    i+=1
   
submision = pd.DataFrame(test_images,columns=['file_name'])

test_batches = test_datagen.flow_from_dataframe(
    dataframe=submision,
    directory=test_path,
    x_col="file_name",
    batch_size=10,
    class_mode=None,
    shuffle=False,
    target_size=(img_size,img_size)
)

predictions = ensemble_model.predict(test_batches,verbose=1)

preds = np.argmax(predictions, axis=-1) 
print(preds)

submision['class_id'] = preds
submision.to_csv('submision39.csv',index=False)